In [1]:
import numpy as np
import tensorflow as tf
import random

In [38]:
def build_network(n_inputs, n_hiddens, n_outputs, learning_rate=0.01):
    x = tf.placeholder(tf.float32, [None, n_inputs], name='inputs')
    y = tf.placeholder(tf.float32, [None, n_outputs], name='outputs')
    
    weights_0_1 = tf.Variable(tf.random_normal([n_inputs, n_hiddens]))
    weights_1_2 = tf.Variable(tf.random_normal([n_hiddens, n_outputs]))
    
    hidden_input = tf.matmul(x, weights_0_1)
    hidden_output = tf.nn.sigmoid(hidden_input)
    
    final_input = tf.matmul(hidden_output, weights_1_2)
    final_output = tf.nn.sigmoid(final_input)

    cost = tf.square(tf.subtract(y, final_output))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    return x, y, final_output, cost, optimizer

    
    

In [39]:
def train_test_split(features, labels, test_size=0.25, random_state=1):
    
    assert(len(features) == len(labels))
    assert(len(features) > 1)
    combined = list(zip(features, labels))
    random.seed(random_state)
    random.shuffle(combined)
    
    features[:], labels[:] = zip(*combined)
    
    split_idx = int(np.ceil(len(features) * test_size))
    
    features_train, features_test, labels_train, labels_test = \
        features[split_idx:], features[:split_idx], labels[split_idx:], labels[:split_idx]
    
    return features_train, features_test, labels_train, labels_test

In [40]:
def accuracy_score(logits, labels):
    assert(len(logits) == len(labels))
    assert(len(logits) > 0)
    
    correct = 0
    
    for logit, label in zip(logits, labels):
        if logit == label:
            correct += 1
    return correct/len(logits)

In [41]:
n_inputs = 6
n_hiddens = 4
n_outputs = 1 # don't change
function = lambda features: np.sum(features) > 2
learning_rate = 0.01
epochs = 10000
test_split = 0.10


In [43]:
# Generate data
features = np.array(np.meshgrid( *([[0, 1]]*n_inputs) )).T.reshape(-1,n_inputs)
labels = np.zeros((features.shape[0], n_outputs), dtype=int)
for key, row in enumerate(features):
    if (function(row)): # Qualifier
        labels[key] = 1

# Split Data
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=test_split, random_state=42)

x, y, pred, cost, optimizer = build_network(n_inputs, n_hiddens, n_outputs, learning_rate)

accuracy = tf.contrib.metrics.accuracy(tf.cast(tf.round(pred), dtype=tf.int32), tf.cast(tf.round(y), dtype=tf.int32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    
    for epoch in range(epochs):
        _, c = sess.run([optimizer, cost], feed_dict={x: features_train,
                                                      y: labels_train})
    
    accuracy = sess.run(accuracy, feed_dict={x: features_test, 
                                        y: labels_test})
    
    print('Accuracy: {}'.format(accuracy))
    
        

starting session...
Accuracy: 1.0
